# 将单机模型训练代码迁移 SecretFlow 联邦学习训练代码教程

## 引言
### 背景
随着数据隐私问题日益受到重视，并且实际业务场景的需求不断演变，联邦学习作为一种特殊的深度学习形式开始迅速兴起。它以一种创新的方式解决了传统集中式训练的数据隐私问题，能够在保护用户隐私的前提下，有效地训练机器学习模型。  
那么假如现实的业务中已经有了单机模型，应该如何将它进行联邦化呢？为了易用性，我们的隐语在设计之初就希望能够以最低的成本来帮助用户迁移已有的单机模型到联邦模型。

### 教程目标以及内容
通过本教程的学习：
1. 读者可以快速的将已有的单机模型正确的使用SecretFlow来进行联邦化。  
2. 先通过单机模型完成模型开发，再进行联邦化适配，也是一个比较推荐的开发实践，可以提高联邦模型的开发效率。

本教程将手把手的带你学习如何将已有的单机模型训练代码迁移到SecretFlow中进行联邦学习。
1. 介绍迁移的整体流程
2. 通过案例介绍在Tensorflow作为后端的迁移流程
3. 通过案例介绍在Pytorch作为后端的迁移流程  


## 迁移的步骤

基于 PyTorch 从单机模型到联邦学习模型，主要需要添加或修改以下几部分
- 添加联邦学习中的参与方
- 修改数据集的处理逻辑
- 修改模型的继承类
- 根据需要决定是否对 metric 、 optimizer 和 loss fuction 进行包装，并使用包装后的函数

基于 TensorFlow 从单机模型到联邦学习模型，主要需要添加或修改以下几部分
- 添加联邦学习中的参与方
- 修改数据集的处理逻辑
- 进行模型的封装

得益于隐语的封装，使用者不需要自己进行大量的代码编写，只需要调用 Secretflow 中的函数，即可便捷完成模型的定义和使用等操作。  
迁移完成后，不同后端不同的模型都使用一套API来进行`fit`,`predict`,`evaluate`等等

## 前期数据准备
相关文档可以参考相关IO文档

下载数据集并解压

In [1]:
import os
import requests
import tarfile
import tempfile

# # Create a temporary folder
# _temp_dir = "/home/beng003/python_project/sf-test/data"

# # Download file
# url = "https://secretflow-data.oss-accelerate.aliyuncs.com/datasets/tf_flowers/flower_photos.tgz"
# save_path = os.path.join(_temp_dir, "flower_photos.tgz")

# response = requests.get(url)
# with open(save_path, "wb") as f:
#     f.write(response.content)

# # Extract the file
# extract_folder = os.path.join(_temp_dir, "flower_photos")
# os.makedirs(extract_folder, exist_ok=True)

# with tarfile.open(save_path, "r:gz") as tar:
#     tar.extractall(path=extract_folder)

# path_to_flower_dataset = extract_folder

path_to_flower_dataset = (
    "/home/beng003/python_project/sf-test/data/datasets/flower_photos"
)

## 基于PyTorch的迁移教程

### 模型在 PyTorch 下的单机模型实现
首先我们给出单机模式下，基于 PyTorch 定义和训练神经网络模型的过程。

In [2]:
import math

import numpy as np
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms

# parameter
batch_size = 5
shuffle = True
random_seed = 1234
train_split = 0.8

# 构建 PyTorch 中 Dataloader 对象
flower_transform = transforms.Compose(
    [
        transforms.Resize((180, 180)),
        transforms.ToTensor(),
    ]
)
flower_dataset = datasets.ImageFolder(
    path_to_flower_dataset, transform=flower_transform
)
dataset_size = len(flower_dataset)
# Define sampler

indices = list(range(dataset_size))
if shuffle:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
split = int(np.floor(train_split * dataset_size))
train_indices, val_indices = indices[:split], indices[split:]
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

# Define databuilder
train_loader = DataLoader(flower_dataset, batch_size=batch_size, sampler=train_sampler)
valid_loader = DataLoader(flower_dataset, batch_size=batch_size, sampler=valid_sampler)


# 定义单机模型结构
import torch
from torch import nn


class ConvRGBNet_torch(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.network = nn.Sequential(
            nn.Conv2d(
                in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1
            ),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Flatten(),
            nn.Linear(16 * 45 * 45, 128),
            nn.ReLU(),
            nn.Linear(128, 5),
        )

    def forward(self, xb):
        return self.network(xb)


# 定义优化器
# initialize
model_torch = ConvRGBNet_torch()

# Define the loss function
loss_model_torch = nn.CrossEntropyLoss()

# Define the optimizer
optimizer_model_torch = torch.optim.SGD(params=model_torch.parameters(), lr=0.01)
optimizer_model_torch.zero_grad()

In [7]:
train_step_per_epoch = len(train_loader)
train_step_per_epoch

4

In [8]:
train_loader

In [21]:
# 获取一个批次的数据
images, labels = next(iter(train_loader))

# 查看图像和标签的形状
print(images.shape)
print(labels.shape)

# 查看第一个批次的图像和标签内容
print(images)
print(labels)

torch.Size([5, 3, 180, 180])
torch.Size([5])
tensor([[[[0.6196, 0.6196, 0.6510,  ..., 0.2431, 0.2627, 0.2431],
          [0.6118, 0.6275, 0.6667,  ..., 0.2118, 0.2275, 0.2275],
          [0.6235, 0.6549, 0.6941,  ..., 0.1804, 0.1961, 0.2000],
          ...,
          [0.5451, 0.5451, 0.5412,  ..., 0.4314, 0.4353, 0.4353],
          [0.5490, 0.5451, 0.5451,  ..., 0.4157, 0.4235, 0.4235],
          [0.5490, 0.5451, 0.5490,  ..., 0.4078, 0.4157, 0.4157]],

         [[0.6039, 0.6078, 0.6431,  ..., 0.3843, 0.4039, 0.3843],
          [0.5961, 0.6157, 0.6588,  ..., 0.3451, 0.3608, 0.3490],
          [0.6078, 0.6392, 0.6824,  ..., 0.2902, 0.3020, 0.3020],
          ...,
          [0.4706, 0.4706, 0.4667,  ..., 0.3647, 0.3647, 0.3608],
          [0.4667, 0.4667, 0.4667,  ..., 0.3569, 0.3647, 0.3647],
          [0.4667, 0.4627, 0.4667,  ..., 0.3569, 0.3647, 0.3686]],

         [[0.6627, 0.6627, 0.6980,  ..., 0.1137, 0.1333, 0.1176],
          [0.6549, 0.6745, 0.7176,  ..., 0.1059, 0.1216, 0.1137

### 基于 PyTorch 的隐语联邦学习模型迁移
基于 PyTorch 从单机模型到联邦学习模型，主要包含以下步骤：
- 添加联邦学习中的参与方
- 修改数据集的处理逻辑
- 修改模型的继承类
- 根据需要决定是否对 metric 、 optimizer 和 loss fuction 进行包装，并使用包装后的函数

接下来，我们将结合实际代码具体讲解这些步骤。

#### 环境设置
添加联邦学习中的参与方，并对各个参与方进行初始化

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
# bob.py
import secretflow as sf

# Check the version of your SecretFlow
print("The version of SecretFlow: {}".format(sf.__version__))

# In case you have a running secretflow runtime already.
sf.shutdown()

pyu_port = 16307
spu_port = 11666

cluster_config = {
    "parties": {
        "alice": {
            # replace with alice's real address.
            "address": "ecm-01:" + str(pyu_port),
            "listen_addr": "0.0.0.0:" + str(pyu_port),
        },
        "bob": {
            # replace with bob's real address.
            "address": "ecm-02:" + str(pyu_port),
            "listen_addr": "0.0.0.0:" + str(pyu_port),
        },
    },
    "self_party": "bob",
}

tls_config = {
    "ca_cert": "/home/beng003/certificate/alice_ca.crt",
    "cert": "/home/beng003/certificate/bob_server_cert.crt",
    "key": "/home/beng003/certificate/bob_server_key.key",
}


sf.init(address="ecm-02:6379", cluster_config=cluster_config, tls_config=tls_config)
alice, bob = sf.PYU("alice"), sf.PYU("bob")

2024-08-13 20:27:07,100	INFO worker.py:1540 -- Connecting to existing Ray cluster at address: ecm-02:6379...
2024-08-13 20:27:07,109	INFO worker.py:1724 -- Connected to Ray cluster.
2024-08-13 20:27:07.133 INFO api.py:233 [bob] -- [Anonymous_job] Started rayfed with {'CLUSTER_ADDRESSES': {'alice': 'ecm-01:16307', 'bob': '0.0.0.0:16307'}, 'CURRENT_PARTY_NAME': 'bob', 'TLS_CONFIG': {'ca_cert': '/home/beng003/certificate/alice_ca.crt', 'cert': '/home/beng003/certificate/bob_server_cert.crt', 'key': '/home/beng003/certificate/bob_server_key.key'}}


The version of SecretFlow: 1.8.0b0


2024-08-13 20:27:08.137 INFO barriers.py:284 [bob] -- [Anonymous_job] Succeeded to create receiver proxy actor.
(ReceiverProxyActor pid=335916) 2024-08-13 20:27:08.132 INFO grpc_proxy.py:359 [bob] -- [Anonymous_job] ReceiverProxy binding port 16307, options: (('grpc.enable_retries', 1), ('grpc.so_reuseport', 0), ('grpc.max_send_message_length', 524288000), ('grpc.max_receive_message_length', 524288000), ('grpc.service_config', '{"methodConfig": [{"name": [{"service": "GrpcService"}], "retryPolicy": {"maxAttempts": 5, "initialBackoff": "5s", "maxBackoff": "30s", "backoffMultiplier": 2, "retryableStatusCodes": ["UNAVAILABLE"]}}]}'))...
(ReceiverProxyActor pid=335916) 2024-08-13 20:27:08.136 INFO grpc_proxy.py:379 [bob] -- [Anonymous_job] Successfully start Grpc service with credentials.
2024-08-13 20:27:09.088 INFO barriers.py:333 [bob] -- [Anonymous_job] SenderProxyActor has successfully created.
2024-08-13 20:27:09.090 INFO barriers.py:520 [bob] -- [Anonymous_job] Try ping ['alice'] at

(raylet) [2024-08-13 20:27:15,357 E 265544 265570] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-13_19-19-00_433060_265452 is over 95% full, available space: 4209774592; capacity: 105089261568. Object creation will fail if spilling is required.
(raylet) [2024-08-13 20:27:25,368 E 265544 265570] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-13_19-19-00_433060_265452 is over 95% full, available space: 4205883392; capacity: 105089261568. Object creation will fail if spilling is required.
(raylet) [2024-08-13 20:27:35,375 E 265544 265570] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-13_19-19-00_433060_265452 is over 95% full, available space: 4209680384; capacity: 105089261568. Object creation will fail if spilling is required.
(raylet) [2024-08-13 20:27:45,383 E 265544 265570] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-13_19-19-00_433060_265452 is over 95% full, available space: 4209717248; capacity: 105089261568. Object 

(ActorPYUFedAvgW pid=336904) {'train-loss': 0.885633647441864, 'train_multiclassaccuracy': tensor(0.6500), 'train_multiclassprecision': tensor(0.6500), 'val_eval_multiclassaccuracy': tensor(0.), 'val_eval_multiclassprecision': tensor(0.)}


(raylet) [2024-08-13 20:37:25,812 E 265544 265570] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-13_19-19-00_433060_265452 is over 95% full, available space: 4207427584; capacity: 105089261568. Object creation will fail if spilling is required.
(raylet) [2024-08-13 20:37:35,819 E 265544 265570] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-13_19-19-00_433060_265452 is over 95% full, available space: 4203675648; capacity: 105089261568. Object creation will fail if spilling is required.
(raylet) [2024-08-13 20:37:45,837 E 265544 265570] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-13_19-19-00_433060_265452 is over 95% full, available space: 4207370240; capacity: 105089261568. Object creation will fail if spilling is required.
(raylet) [2024-08-13 20:37:55,851 E 265544 265570] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-13_19-19-00_433060_265452 is over 95% full, available space: 4203499520; capacity: 105089261568. Object 

#### 封装单机模式下的数据处理逻辑
同样地，在联邦学习中我们也需要对数据进行预处理，使之符合模型的输入，所以参考在[SecretFlow 中使用自定义 DataBuilder (Torch)构建 dataset builder](https://www.secretflow.org.cn/docs/secretflow/latest/zh-Hans/tutorial/CustomDataLoaderTorch)，我们选择文件夹路径作为参数，并且封装单机模式下的数据处理逻辑，最后返回 (data_set，steps_per_epoch)的结果，封装代码如下：

In [5]:
def create_dataset_builder(
    batch_size=5,
    train_split=0.8,
    shuffle=True,
    random_seed=1234,
):
    def dataset_builder(x, stage="train"):
        """ """
        ######################################单机模型数据读取代码#########################################
        import math

        import numpy as np
        from torch.utils.data import DataLoader
        from torch.utils.data.sampler import SubsetRandomSampler
        from torchvision import datasets, transforms

        # Define dataset

        flower_transform = transforms.Compose(
            [
                transforms.Resize((180, 180)),
                transforms.ToTensor(),
            ]
        )
        flower_dataset = datasets.ImageFolder(x, transform=flower_transform)
        dataset_size = len(flower_dataset)
        # Define sampler

        indices = list(range(dataset_size))
        if shuffle:
            np.random.seed(random_seed)
            np.random.shuffle(indices)
        split = int(np.floor(train_split * dataset_size))
        train_indices, val_indices = indices[:split], indices[split:]
        train_sampler = SubsetRandomSampler(train_indices)
        valid_sampler = SubsetRandomSampler(val_indices)

        # Define databuilder
        train_loader = DataLoader(
            flower_dataset, batch_size=batch_size, sampler=train_sampler
        )
        valid_loader = DataLoader(
            flower_dataset, batch_size=batch_size, sampler=valid_sampler
        )
        #############################################################################################
        # Return
        if stage == "train":
            train_step_per_epoch = len(train_loader)

            return train_loader, train_step_per_epoch
        elif stage == "eval":
            eval_step_per_epoch = len(valid_loader)
            return valid_loader, eval_step_per_epoch

    return dataset_builder

#### 构建 dataset_builder_dict

我们通过 dataset_builder_dict 为各个参与方传入封装数据处理逻辑的 create_dataset_builder 函数的参数。

In [6]:
# prepare dataset dict
data_builder_dict = {
    alice: create_dataset_builder(
        batch_size=5,
        train_split=0.8,
        shuffle=False,
        random_seed=1234,
    ),
    bob: create_dataset_builder(
        batch_size=5,
        train_split=0.8,
        shuffle=False,
        random_seed=1234,
    ),
}

#### 在隐语的框架下定义基于 PyTorch 的模型架构
参考 PyTorch 单机模式下的模型，我们在隐语的框架下定义同样结构的模型。

我们只需修改继承类将 **torch.nn.Module** 改为 **secretflow.ml.nn.core.torch.BaseModule**，就可以完成模型架构的定义。

从迁移过程可以看出，将单机模型在隐语框架下进行定义所进行的代码改动非常小，整体迁移非常方便，充分展现了隐语框架的易用性。

In [7]:
from secretflow.ml.nn.core.torch import BaseModule


######################################单机模型代码#########################################
class ConvRGBNet(BaseModule):  # 只有这里有变化，其他和单机模型定义保持一致
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.network = nn.Sequential(
            nn.Conv2d(
                in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1
            ),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Flatten(),
            nn.Linear(16 * 45 * 45, 128),
            nn.ReLU(),
            nn.Linear(128, 5),
        )

    def forward(self, xb):
        return self.network(xb)


################################################################################

#### 构建TorchModel
`TorchModel`是我们在隐语中定义的一个概念，层级概念上对应`keras model`。目的是将将用户定义的`模型`，`loss函数`,`optimizer`,`metrics`封装在一起，用与后续FLModel在启动后统一多后端逻辑。  
其中需要注意的是，我们的loss_fn，optim_fn，metrics都需要通过一个wrapper封装成一个偏函数传入进去，在后面会有详细的解析。

In [8]:
from secretflow.ml.nn import FLModel
from secretflow.security.aggregation import SecureAggregator
from torch import nn, optim
from torchmetrics import Accuracy, Precision
from secretflow.ml.nn.core.torch import TorchModel, metric_wrapper, optim_wrapper


device_list = [alice, bob]
aggregator = SecureAggregator(alice, [alice, bob])
# prepare model
num_classes = 5


input_shape = (180, 180, 3)
# torch model

optim_fn = optim_wrapper(optim.Adam, lr=1e-3)
model_def = TorchModel(
    model_fn=ConvRGBNet,
    loss_fn=nn.CrossEntropyLoss,
    optim_fn=optim_fn,
    metrics=[
        metric_wrapper(
            Accuracy, task="multiclass", num_classes=num_classes, average='micro'
        ),
        metric_wrapper(
            Precision, task="multiclass", num_classes=num_classes, average='micro'
        ),
    ],
)

2024-08-13 20:27:46.045 INFO proxy.py:187 [bob] -- [Anonymous_job] Create proxy actor <class 'secretflow.device.proxy.Actor_Masker'> with party alice.
2024-08-13 20:27:46.046 INFO proxy.py:187 [bob] -- [Anonymous_job] Create proxy actor <class 'secretflow.device.proxy.Actor_Masker'> with party bob.


#### 定义FLModel

In [9]:
fed_model = FLModel(
    device_list=device_list,
    model=model_def,
    aggregator=aggregator,
    backend="torch",  # backend support ['tensorflow', 'torch']
    strategy="fed_avg_w",
    random_seed=1234,
)

2024-08-13 20:27:53.221 INFO proxy.py:187 [bob] -- [Anonymous_job] Create proxy actor <class 'abc.ActorPYUFedAvgW'> with party alice.
2024-08-13 20:27:53.223 INFO proxy.py:187 [bob] -- [Anonymous_job] Create proxy actor <class 'abc.ActorPYUFedAvgW'> with party bob.


In [10]:
data = {
    alice: path_to_flower_dataset,
    bob: path_to_flower_dataset,
}
history = fed_model.fit(
    data,
    None,
    validation_data=data,
    epochs=5,
    batch_size=5,
    aggregate_freq=2,
    sampler_method="batch",
    random_seed=1234,
    dp_spent_step_freq=1,
    dataset_builder=data_builder_dict,
)

2024-08-13 20:27:58.160 INFO fl_model.py:396 [bob] -- [Anonymous_job] FL Train Params: {'x': {PYURuntime(alice): '/home/beng003/python_project/sf-test/data/datasets/flower_photos', PYURuntime(bob): '/home/beng003/python_project/sf-test/data/datasets/flower_photos'}, 'y': None, 'batch_size': 5, 'batch_sampling_rate': None, 'epochs': 5, 'verbose': 1, 'callbacks': None, 'validation_data': {PYURuntime(alice): '/home/beng003/python_project/sf-test/data/datasets/flower_photos', PYURuntime(bob): '/home/beng003/python_project/sf-test/data/datasets/flower_photos'}, 'shuffle': False, 'class_weight': None, 'sample_weight': None, 'validation_freq': 1, 'aggregate_freq': 2, 'label_decoder': None, 'max_batch_size': 20000, 'prefetch_buffer_size': None, 'sampler_method': 'batch', 'random_seed': 1234, 'dp_spent_step_freq': 1, 'audit_log_dir': None, 'dataset_builder': {PYURuntime(alice): <function create_dataset_builder.<locals>.dataset_builder at 0x7fe4d14b3640>, PYURuntime(bob): <function create_datase

Train Processing: :   0%|          | 0/4 [00:00<?, ?it/s]

Epoch 1/5


/home/beng003/anaconda/envs/sf/lib/python3.10/site-packages/secretflow/ml/nn/metrics.py:59: UserWarning: Please pay attention to local metrics, global only do naive aggregation.
  warnings.warn(
2024-08-13 20:30:37.843189: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Train Processing: :   0%|          | 0/4 [00:00<?, ?it/s]

Epoch 2/5


...

Train Processing: :   0%|          | 0/4 [00:00<?, ?it/s]

Epoch 5/5


(ActorPYUFedAvgW pid=336904) {'train-loss': 1.0411512851715088, 'train_multiclassaccuracy': tensor(0.7000), 'train_multiclassprecision': tensor(0.7000), 'val_eval_multiclassaccuracy': tensor(0.), 'val_eval_multiclassprecision': tensor(0.)}


(raylet) [2024-08-13 20:35:45,736 E 265544 265570] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-13_19-19-00_433060_265452 is over 95% full, available space: 4207837184; capacity: 105089261568. Object creation will fail if spilling is required.
(raylet) [2024-08-13 20:35:55,744 E 265544 265570] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-13_19-19-00_433060_265452 is over 95% full, available space: 4206784512; capacity: 105089261568. Object creation will fail if spilling is required.
(raylet) [2024-08-13 20:36:05,751 E 265544 265570] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-13_19-19-00_433060_265452 is over 95% full, available space: 4207751168; capacity: 105089261568. Object creation will fail if spilling is required.
(raylet) [2024-08-13 20:36:15,758 E 265544 265570] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-13_19-19-00_433060_265452 is over 95% full, available space: 4203868160; capacity: 105089261568. Object 

Train Processing: :  50%|█████     | 2/4 [01:40<01:40, 50.01s/it, {'multiclassaccuracy': 0.65, 'multiclassprecision': 0.65, 'val_multiclassaccuracy': 0.0, 'val_multiclassprecision': 0.0}]


In [11]:
history

{'global_history': {'multiclassaccuracy': [0.2, 0.4, 0.6, 0.7, 0.65],
  'multiclassprecision': [0.2, 0.4, 0.6, 0.7, 0.65],
  'val_multiclassaccuracy': [0.0, 0.0, 0.0, 0.0, 0.0],
  'val_multiclassprecision': [0.0, 0.0, 0.0, 0.0, 0.0]},
 'local_history': {'alice_train-loss': [1.5570476055145264,
   1.4086425304412842,
   1.2114331722259521,
   1.0411512851715088,
   0.885633647441864],
  'alice_train_multiclassaccuracy': [tensor(0.2000),
   tensor(0.4000),
   tensor(0.6000),
   tensor(0.7000),
   tensor(0.6500)],
  'alice_train_multiclassprecision': [tensor(0.2000),
   tensor(0.4000),
   tensor(0.6000),
   tensor(0.7000),
   tensor(0.6500)],
  'alice_val_eval_multiclassaccuracy': [tensor(0.),
   tensor(0.),
   tensor(0.),
   tensor(0.),
   tensor(0.)],
  'alice_val_eval_multiclassprecision': [tensor(0.),
   tensor(0.),
   tensor(0.),
   tensor(0.),
   tensor(0.)],
  'bob_train-loss': [1.5570476055145264,
   1.4086425304412842,
   1.2114331722259521,
   1.0411512851715088,
   0.8856336474

### 构建完成

得到FLModel对象之后，就可以用这个对象进行`fit`,`evaluate`,`predict`等操作了。

## Why need wraps

### 将单机模型下的 optimizer 包装（wrap）
对 optimizer 进行包装的原因：我们需要使用 optimizer 在训练过程中自动地调整模型参数，以使模型在给定的训练数据上达到最佳的性能表现。同样地，联邦学习也可以通过 optimizer 实现模型更好的性能。但由于隐语的设计机制需要把相关模块通过序列化到具体的机器上才会执行，因此在优化器需要指定参数时，需要做一次封装。

通过`optim_wrapper`进行优化器（optimizer）的包装，并且通过追根溯源，我们可以看到。
```python
from secretflow.ml.nn.core.torch import optim_wrapper
```
[source code](https://github.com/secretflow/secretflow/blob/main/secretflow/ml/nn/core/torch/utils.py#L23)

In [16]:
def optim_wrapper(func, *args, **kwargs):

    def wrapped_func(params):
        return func(params, *args, **kwargs)

    return wrapped_func

可以看到函数实际上都是通过传入一个需要包装的函数名称，位置参数和关键字参数对函数完成包装，通过关键字参数确保了指定的参数赋值，然后返回包装好的函数。
因此
```python
optim_fn = optim_wrapper(optim.Adam, lr=1e-2)
```
实际上相当于调用
```python
optim.Adam(lr=1e-2)
```

### 将单机模型下的 metric 包装（wrap）
对 metric 进行包装的原因:metric 在机器学习和深度学习中用于衡量模型的性能和表现，它们是评估模型在训练、验证或测试数据上的效果的标准。同样地，我们也希望使用 metric 衡量联邦学习模型的性能和表现。但由于隐语的设计机制同 PyTorch 有些差异，因此在衡量指标需要指定参数时，需要做一次封装来保证两者的一致性。

通过`metric_wrapper`进行衡量指标（metric）的包装，并且通过追根溯源，我们可以看到
```python
from secretflow.ml.nn.core.torch import metric_wrapper
```
[source code](https://github.com/secretflow/secretflow/blob/main/secretflow/ml/nn/core/torch/utils.py#L23)

In [12]:
def metric_wrapper(func, *args, **kwargs):
    def wrapped_func():
        return func(*args, **kwargs)

    return wrapped_func

可以看到函数实际上是通过传入一个需要包装的函数名称，位置参数和关键字参数对函数完成包装，通过关键字参数确保了指定的参数赋值，然后返回包装好的函数；与优化器包装类似
因此
```python
metric_wrapper(Accuracy, task="multiclass", num_classes=10, average='micro')
```
实际上相当于调用
```python
Accuracy(task="multiclass", num_classes=10, average='micro')

### 将单机模型下的 loss function 包装（wrap）
参考 `optim_wrapper` 和 `metric_wrapper` 的定义方式，自定义 `loss_function_wrapper`对损失函数（loss function）进行包装。

如果模型使用损失函数的默认参数，则不需要使用包装

In [13]:
def loss_function_wrapper(func, *args, **kwargs):
    def wrapped_func():
        return func(*args, **kwargs)

    return wrapped_func

得益于隐语的封装，并且根据前述的包装原理可知，我们实际上需要通过包装完成一个参数具体化的函数，所以在这里 `nn.CrossEntropyLoss` 是需要包装的函数，并且其参数取值，就是需要传入的参数值。因为在其默认参数取值设置中，  `reduction='mean'` , 此处我们试着将其修改为 ` reduction='sum'`。包装损失函数只需要写成：

In [14]:
loss_wrapper = loss_function_wrapper(nn.CrossEntropyLoss, reduction='sum')

### 对单机模型使用包装（wrap）后的优化器（optimizer）、衡量指标（metric）和损失函数（loss function）
在隐语框架下，使用者可以根据需要选择是否对优化器（optimizer）、衡量指标（metric）和损失函数（loss function）进行包装，从而更好地训练联邦学习模型，充分发挥隐语框架的灵活性。

## 小结
基于 PyTorch 从单机模型到联邦学习模型，主要需要添加或修改以下几部分
- 添加联邦学习中的参与方
- 修改数据集的处理逻辑
- 修改模型的继承类
- 根据需要决定是否对 metric 、 optimizer 和 loss fuction 进行包装，并使用包装后的函数

得益于隐语的封装，使用者不需要自己完成模型定义等代码的编写，只需要调用 Secretflow 中的函数，即可便捷完成模型的定义和使用等操作。

## 基于 TensorFlow 的迁移教程
### 模型在 TensorFlow 下的单机模型实现
首先我们给出单机模式下，基于 TensorFlow 定义和训练神经网络模型的过程。对于数据，我们将其加载成 TensorFlow 的 dataset 对象


In [12]:
import math
import tensorflow as tf

img_height = 180
img_width = 180
batch_size = 5
# In this example, we use the TensorFlow interface for development.
data_set = tf.keras.utils.image_dataset_from_directory(
    path_to_flower_dataset,
    validation_split=0.2,
    subset="both",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)
train_set = data_set[0]
test_set = data_set[1]

from tensorflow import keras

# Create model
num_classes = 5
input_shape = (180, 180, 3)
total_epochs = 10

model_tensorflow = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        tf.keras.layers.Rescaling(1.0 / 255),
        tf.keras.layers.Conv2D(32, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(32, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(32, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(num_classes),
    ]
)

# Compile model
model_tensorflow.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=["accuracy"],
)

# Model training and validation
print('-------Start training-------')
history = model_tensorflow.fit(
    train_set,
    validation_data=test_set,
    batch_size=batch_size,
    epochs=total_epochs,
    verbose=True,
    shuffle=True,
)

Found 25 files belonging to 5 classes.
Using 20 files for training.
Using 5 files for validation.
-------Start training-------
Epoch 1/10
4/4 [==============================] - 1s 128ms/step - loss: 1.7442 - accuracy: 0.2000 - val_loss: 1.5286 - val_accuracy: 0.6000
Epoch 2/10
4/4 [==============================] - 0s 75ms/step - loss: 1.2989 - accuracy: 0.5500 - val_loss: 1.4937 - val_accuracy: 0.4000
Epoch 3/10
4/4 [==============================] - 0s 67ms/step - loss: 0.9715 - accuracy: 0.7000 - val_loss: 1.4600 - val_accuracy: 0.2000
Epoch 4/10
4/4 [==============================] - 0s 68ms/step - loss: 0.6000 - accuracy: 0.8500 - val_loss: 1.9802 - val_accuracy: 0.2000
Epoch 5/10
4/4 [==============================] - 0s 81ms/step - loss: 0.3277 - accuracy: 0.8500 - val_loss: 1.5714 - val_accuracy: 0.4000
Epoch 6/10
4/4 [==============================] - 0s 78ms/step - loss: 0.2519 - accuracy: 0.9500 - val_loss: 1.5647 - val_accuracy: 0.2000
Epoch 7/10
4/4 [======================

### 基于 TensorFlow 的隐语联邦学习模型迁移
#### 概述 
基于 TensorFlow 从单机模型到联邦学习模型，主要包含以下步骤：
- 添加联邦学习中的参与方
- 修改数据集的处理逻辑
- 进行模型的封装

接下来，我们将结合实际代码具体讲解这些步骤。
### 环境设置
添加联邦学习中的参与方，并初始化各个参与方

In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# bob.py
import secretflow as sf

# Check the version of your SecretFlow
print("The version of SecretFlow: {}".format(sf.__version__))

# In case you have a running secretflow runtime already.
sf.shutdown()

pyu_port = 16307
spu_port = 11666

cluster_config = {
    "parties": {
        "alice": {
            # replace with alice's real address.
            "address": "ecm-01:" + str(pyu_port),
            "listen_addr": "0.0.0.0:" + str(pyu_port),
        },
        "bob": {
            # replace with bob's real address.
            "address": "ecm-02:" + str(pyu_port),
            "listen_addr": "0.0.0.0:" + str(pyu_port),
        },
    },
    "self_party": "bob",
}

tls_config = {
    "ca_cert": "/home/beng003/certificate/alice_ca.crt",
    "cert": "/home/beng003/certificate/bob_server_cert.crt",
    "key": "/home/beng003/certificate/bob_server_key.key",
}


sf.init(address="ecm-02:6379", cluster_config=cluster_config, tls_config=tls_config)
alice, bob = sf.PYU("alice"), sf.PYU("bob")

The version of SecretFlow: 1.8.0b0


2024-08-13 20:38:18.435 INFO api.py:342 [bob] -- [Anonymous_job] Shutdowning rayfed intendedly...
2024-08-13 20:38:18.437 INFO api.py:356 [bob] -- [Anonymous_job] Wait for data sending.
2024-08-13 20:38:18.438 INFO message_queue.py:72 [bob] -- [Anonymous_job] Notify message polling thread[DataSendingQueueThread] to exit.
2024-08-13 20:38:18.440 INFO message_queue.py:102 [bob] -- [Anonymous_job] The message polling thread[DataSendingQueueThread] was exited.
2024-08-13 20:38:18.441 INFO message_queue.py:72 [bob] -- [Anonymous_job] Notify message polling thread[ErrorSendingQueueThread] to exit.
2024-08-13 20:38:18.491 INFO message_queue.py:102 [bob] -- [Anonymous_job] The message polling thread[ErrorSendingQueueThread] was exited.
2024-08-13 20:38:18.492 INFO barriers.py:469 [bob] -- [Anonymous_job] Stop sender proxy actor...
2024-08-13 20:38:18.496 INFO barriers.py:473 [bob] -- [Anonymous_job] Sender proxy actor stoped.
2024-08-13 20:38:18.496 INFO api.py:384 [bob] -- [Anonymous_job] Shu

(raylet) [2024-08-13 20:38:25,876 E 265544 265570] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-13_19-19-00_433060_265452 is over 95% full, available space: 4206067712; capacity: 105089261568. Object creation will fail if spilling is required.
(raylet) [2024-08-13 20:38:35,883 E 265544 265570] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-13_19-19-00_433060_265452 is over 95% full, available space: 4207079424; capacity: 105089261568. Object creation will fail if spilling is required.
(raylet) [2024-08-13 20:38:45,891 E 265544 265570] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-13_19-19-00_433060_265452 is over 95% full, available space: 4203225088; capacity: 105089261568. Object creation will fail if spilling is required.
(pid=349295) 2024-08-13 20:38:53.044216: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
(raylet) [2024-08-13 20:38:55,902 E 265544 265570] (raylet) file_system_monitor.

(raylet) [2024-08-13 20:40:55,997 E 265544 265570] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-13_19-19-00_433060_265452 is over 95% full, available space: 4202708992; capacity: 105089261568. Object creation will fail if spilling is required.
(raylet) [2024-08-13 20:41:06,004 E 265544 265570] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-13_19-19-00_433060_265452 is over 95% full, available space: 4206493696; capacity: 105089261568. Object creation will fail if spilling is required.
(raylet) [2024-08-13 20:41:16,011 E 265544 265570] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-13_19-19-00_433060_265452 is over 95% full, available space: 4202614784; capacity: 105089261568. Object creation will fail if spilling is required.
(raylet) [2024-08-13 20:41:26,018 E 265544 265570] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-13_19-19-00_433060_265452 is over 95% full, available space: 4205371392; capacity: 105089261568. Object 

In [15]:
def create_dataset_builder(
    batch_size=5,
):
    def dataset_builder(folder_path, stage="train"):
        ############################单机模型代码#############################################
        import math

        import tensorflow as tf

        img_height = 180
        img_width = 180
        data_set = tf.keras.utils.image_dataset_from_directory(
            folder_path,
            validation_split=0.2,
            subset="both",
            seed=123,
            image_size=(img_height, img_width),
            batch_size=batch_size,
        )
        ####################################################################################
        if stage == "train":
            train_dataset = data_set[0]
            train_step_per_epoch = math.ceil(len(data_set[0].file_paths) / batch_size)
            return train_dataset, train_step_per_epoch
        elif stage == "eval":
            eval_dataset = data_set[1]
            eval_step_per_epoch = math.ceil(len(data_set[1].file_paths) / batch_size)
            return eval_dataset, eval_step_per_epoch

    return dataset_builder

### 构建 dataset_builder_dict
我们通过 dataset_builder_dict 为各个参与方传入封装数据处理逻辑的 create_dataset_builder 函数的参数。

In [16]:
data_builder_dict = {
    alice: create_dataset_builder(
        batch_size=5,
    ),
    bob: create_dataset_builder(
        batch_size=5,
    ),
}

### 在隐语的框架下定义基于 TensorFlow 的模型架构
参考 TensorFlow 单机模式下的模型，我们在隐语的框架下定义同样结构的模型。

如前所述，我们需要使用 optimizer 在训练过程中自动地调整模型参数，以使模型在给定的训练数据上达到最佳的性能表现；使用 metric 在机器学习和深度学习中用于衡量模型的性能和表现；使用损失函数监督神经网络的训练。同样地，联邦学习也可以通过使用优化器（optimizer）、衡量指标（metric）和损失函数（loss function） 实现模型更好的性能。在隐语框架下，优化器（optimizer）、衡量指标（metric）和损失函数（loss function）的使用方法和 TensorFlow 中的使用方法一致，也同样通过模型的 compile 函数实现。

从迁移过程可以看出，代码修改幅度非常小，整体迁移过程非常方便，充分展现了隐语框架的易用性和便捷性。

In [17]:
def create_conv_flower_model(input_shape, num_classes, name='model'):
    def create_model():
        ##########################单机模型代码##################################
        from tensorflow import keras

        # Create model

        model = keras.Sequential(
            [
                keras.Input(shape=input_shape),
                tf.keras.layers.Rescaling(1.0 / 255),
                tf.keras.layers.Conv2D(32, 3, activation='relu'),
                tf.keras.layers.MaxPooling2D(),
                tf.keras.layers.Conv2D(32, 3, activation='relu'),
                tf.keras.layers.MaxPooling2D(),
                tf.keras.layers.Conv2D(32, 3, activation='relu'),
                tf.keras.layers.MaxPooling2D(),
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(128, activation='relu'),
                tf.keras.layers.Dense(num_classes),
            ]
        )
        # Compile model
        model.compile(
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            optimizer='adam',
            metrics=["accuracy"],
        )
        return model
        ##########################单机模型代码##################################

    return create_model

In [18]:
from secretflow.ml.nn import FLModel
from secretflow.security.aggregation import SecureAggregator

In [19]:
device_list = [alice, bob]
aggregator = SecureAggregator(alice, [alice, bob])

# prepare model
num_classes = 5
input_shape = (180, 180, 3)

# keras model
model = create_conv_flower_model(input_shape, num_classes)


fed_model = FLModel(
    device_list=device_list,
    model=model,
    aggregator=aggregator,
    backend="tensorflow",
    strategy="fed_avg_w",
    random_seed=1234,
)

2024-08-13 20:38:46.445 INFO proxy.py:187 [bob] -- [Anonymous_job] Create proxy actor <class 'secretflow.device.proxy.Actor_Masker'> with party alice.
2024-08-13 20:38:46.446 INFO proxy.py:187 [bob] -- [Anonymous_job] Create proxy actor <class 'secretflow.device.proxy.Actor_Masker'> with party bob.


2024-08-13 20:38:50.311 INFO proxy.py:187 [bob] -- [Anonymous_job] Create proxy actor <class 'secretflow.device.proxy.ActorPYUFedAvgW'> with party alice.
2024-08-13 20:38:50.313 INFO proxy.py:187 [bob] -- [Anonymous_job] Create proxy actor <class 'secretflow.device.proxy.ActorPYUFedAvgW'> with party bob.


### 训练和验证模型
传入参与方的数据集路径，进行模型的训练和验证

In [20]:
data = {
    alice: path_to_flower_dataset,
    bob: path_to_flower_dataset,
}
history = fed_model.fit(
    data,
    None,
    validation_data=data,
    epochs=2,
    batch_size=5,
    aggregate_freq=2,
    sampler_method="batch",
    random_seed=1234,
    dp_spent_step_freq=1,
    dataset_builder=data_builder_dict,
)

2024-08-13 20:39:08.338 INFO fl_model.py:396 [bob] -- [Anonymous_job] FL Train Params: {'x': {PYURuntime(alice): '/home/beng003/python_project/sf-test/data/datasets/flower_photos', PYURuntime(bob): '/home/beng003/python_project/sf-test/data/datasets/flower_photos'}, 'y': None, 'batch_size': 5, 'batch_sampling_rate': None, 'epochs': 2, 'verbose': 1, 'callbacks': None, 'validation_data': {PYURuntime(alice): '/home/beng003/python_project/sf-test/data/datasets/flower_photos', PYURuntime(bob): '/home/beng003/python_project/sf-test/data/datasets/flower_photos'}, 'shuffle': False, 'class_weight': None, 'sample_weight': None, 'validation_freq': 1, 'aggregate_freq': 2, 'label_decoder': None, 'max_batch_size': 20000, 'prefetch_buffer_size': None, 'sampler_method': 'batch', 'random_seed': 1234, 'dp_spent_step_freq': 1, 'audit_log_dir': None, 'dataset_builder': {PYURuntime(alice): <function create_dataset_builder.<locals>.dataset_builder at 0x7fe39c5a1870>, PYURuntime(bob): <function create_datase

(ActorPYUFedAvgW pid=349295) Found 25 files belonging to 5 classes.
(ActorPYUFedAvgW pid=349295) Using 20 files for training.
(ActorPYUFedAvgW pid=349295) Using 5 files for validation.


Train Processing: :   0%|          | 0/4 [00:00<?, ?it/s]

Epoch 1/2


(raylet) [2024-08-13 20:39:15,921 E 265544 265570] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-13_19-19-00_433060_265452 is over 95% full, available space: 4203036672; capacity: 105089261568. Object creation will fail if spilling is required.
(raylet) [2024-08-13 20:39:25,928 E 265544 265570] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-13_19-19-00_433060_265452 is over 95% full, available space: 4206841856; capacity: 105089261568. Object creation will fail if spilling is required.
(raylet) [2024-08-13 20:39:35,936 E 265544 265570] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-13_19-19-00_433060_265452 is over 95% full, available space: 4202991616; capacity: 105089261568. Object creation will fail if spilling is required.
(raylet) [2024-08-13 20:39:45,945 E 265544 265570] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-13_19-19-00_433060_265452 is over 95% full, available space: 4205776896; capacity: 105089261568. Object 

...

Train Processing: :   0%|          | 0/4 [00:00<?, ?it/s]

Epoch 2/2


(raylet) [2024-08-13 20:40:15,967 E 265544 265570] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-13_19-19-00_433060_265452 is over 95% full, available space: 4206669824; capacity: 105089261568. Object creation will fail if spilling is required.
(raylet) [2024-08-13 20:40:25,975 E 265544 265570] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-13_19-19-00_433060_265452 is over 95% full, available space: 4202786816; capacity: 105089261568. Object creation will fail if spilling is required.
(raylet) [2024-08-13 20:40:35,982 E 265544 265570] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-13_19-19-00_433060_265452 is over 95% full, available space: 4205568000; capacity: 105089261568. Object creation will fail if spilling is required.
(raylet) [2024-08-13 20:40:45,989 E 265544 265570] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-08-13_19-19-00_433060_265452 is over 95% full, available space: 4206583808; capacity: 105089261568. Object 

Train Processing: :  50%|█████     | 2/4 [00:39<00:39, 19.55s/it, {'loss': 1.5927668, 'accuracy': 0.28, 'val_loss': 1.5951079, 'val_accuracy': 0.2}]


(ActorPYUFedAvgW pid=349295) Found 25 files belonging to 5 classes.
(ActorPYUFedAvgW pid=349295) Using 20 files for training.
(ActorPYUFedAvgW pid=349295) Using 5 files for validation.


In [21]:
history

{'global_history': {'loss': [1.9371605, 1.5927668],
  'accuracy': [0.15, 0.28],
  'val_loss': [1.5980835, 1.5951079],
  'val_accuracy': [0.4, 0.2]},
 'local_history': {'alice_loss': [1.9371605, 1.5927668],
  'alice_accuracy': [0.15, 0.28],
  'alice_val_loss': [1.5980835, 1.5951079],
  'alice_val_accuracy': [0.4, 0.2],
  'bob_loss': [1.9371605, 1.5927668],
  'bob_accuracy': [0.15, 0.28],
  'bob_val_loss': [1.5980835, 1.5951079],
  'bob_val_accuracy': [0.4, 0.2]}}

### 小结
基于 TensorFlow 从单机模型到联邦学习模型，主要需要添加或修改以下几部分
- 添加联邦学习中的参与方
- 修改数据集的处理逻辑
- 进行模型的封装

得益于隐语的封装，使用者不需要自己进行大量的代码编写，只需要调用 Secretflow 中的函数，即可便捷完成模型的定义和使用等操作。

## 总结
本教程说明了使用者能够在 SecretFlow 隐语的框架下，体会到和单机模式下，使用 PyTorch 或 Tensorflow 编程几乎一致的联邦学习模型使用体验。充分展现了隐语框架具有易用性和便捷性等优点。